In [ ]:
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col

def quiet_logs(sc):
  logger = sc._jvm.org.apache.log4j
  logger.LogManager.getLogger("org"). setLevel(logger.Level.ERROR)
  logger.LogManager.getLogger("akka").setLevel(logger.Level.ERROR)

HDFS_NAMENODE = 'hdfs://192.168.0.25:9000'
spark_conf = SparkConf()

#if main application is not executing on the same network with spark-master
#then some additional configuration for spark context is required
spark_conf.setAll([
    ('spark.master', 'spark://127.17.0.1:7077'), 
    ('spark.app.name', 'wordcount'),
    ('spark.submit.deployMode', 'client'),
    ('spark.ui.showConsoleProgress', 'true'),
    ('spark.eventLog.enabled', 'false'),
    ('spark.logConf', 'false'),
    ('spark.driver.bindAddress', '0.0.0.0'),
    ('spark.driver.host', '192.168.0.25'),
])
#in case that main application is executing on the same network with spark-master
#only this line is needed
#conf = SparkConf().setAppName("wordcount").setMaster("spark://spark-master:7077")


sc = SparkContext(conf=spark_conf)
spark = SparkSession(sc)
quiet_logs(spark)

In [ ]:
textFile = spark.sparkContext.textFile(HDFS_NAMENODE+"/dante/divine_comedy.txt")

# Creates a DataFrame having a single column named "line"
df = textFile.map(lambda r: Row(r)).toDF(["line"])
virgil = df.filter(col("line").like("%Virgil%"))
print(virgil.count())
print(virgil.filter(col("line").like("%me%")).count())
print(virgil.filter(col("line").like("%me%")).show())
print(virgil.filter(col("line").like("%me%")).collect())